In [2]:
import pandas as pd
from scipy.cluster.hierarchy import dendrogram, linkage
from geopy.distance import vincenty
from scipy import spatial
import networkx as nx
import matplotlib.pyplot as plt
from functions import *
import Parameters as P

In [3]:
data_all = pd.read_csv('../GIS_data/building_all_null.csv')

In [3]:
in_builing_piping_sf = 0.01

In [4]:
data_all['lat_lon'] = data_all.apply(lambda row: (row['y_lat'], row['x_lon']), axis=1)

In [5]:
if len(data_all)>1000:
    k = 1000
else:
    k = len(data_all)

In [6]:
X_lat_lon = list(data_all['lat_lon'])
query_point = (37.763005, -122.493608)
tree = spatial.KDTree(X_lat_lon)
dist, index_select = tree.query(query_point, k=k)

In [7]:
X_lat_lon_select = [X_lat_lon[i] for i in index_select]
Z = linkage(X_lat_lon_select, 'ward')

In [8]:
data = pd.DataFrame()
for i in index_select:
    select_row = data_all.loc[i]
    data = data.append(select_row)

In [9]:
data = data.reset_index()

In [10]:
clusters = hierarchical_cluster(X_lat_lon_select, Z)
index_0 = data[data['y_lat'] == query_point[0]].index.tolist()[0]

In [11]:
num_buildings = 1
SUM_pop = data.loc[int(index_0)]['SUM_pop']
building_sqft = data.loc[int(index_0)]['Area_m2']
MIN_ELEV = data.loc[int(index_0)]['ELEV_treat']
inbuilding_floors = data.loc[int(index_0)]['num_floor']
total_dist = building_sqft*in_builing_piping_sf
inbuilding_pumping = pump_energy_building(inbuilding_floors)
inbuilding_flow = SUM_pop*0.2
#inbuilding_treatment_energy = 9.5*(inbuilding_flow)**(-0.3)*3.6*0.8
inbuilding_treatment_energy = 8*(inbuilding_flow)**(-0.1)*3.6*0.8
total_energy = inbuilding_pumping+inbuilding_treatment_energy

In [12]:
cluster_points = []
cluster_points.append(index_0)
points_checked = []
X_lat_lon_to_check = []
totals = {'num_buildings':num_buildings, 'SUM_pop':SUM_pop ,'ELEV_treat':MIN_ELEV, 'total_dist':total_dist, 'inbuilding_pumping':inbuilding_pumping, 'total_energy':total_energy }
G = nx.Graph()
points_checked.append(index_0)
for key in clusters.keys():
    if index_0 in clusters[key]:
        add = False
        ids_check = list(set(clusters[key])^set(points_checked))
        for item in ids_check:
            X_lat_lon_to_check.append((X_lat_lon_select[int(item-1)]))
            
        for start in ids_check:
            points_checked.append(int(start))
            G.add_node(int(start))
            for destination in G.nodes():
                dist = vincenty(data.iloc[int(start)]['lat_lon'], data.iloc[int(destination)]['lat_lon']).meters
                G.add_path([item, destination], weight=dist)
            MST_distance = find_MST_distance(G)
            building_floors = data.iloc[int(start)]['num_floor']
            building_sqft = data.iloc[int(start)]['Area_m2']
            piping_distance = MST_distance + building_sqft*in_builing_piping_sf
            conveyance_energy = find_conveyance_energy(data, start, totals, piping_distance, building_floors)
            treatment_energy = find_treatment_energy(data, start, totals)
            treatment_embodied = find_treatment_embodied_energy(data, start, totals, type = False)
            infrastructure = find_infrastructure_energy(data, start, totals, piping_distance)
            total_energy = conveyance_energy + treatment_energy + infrastructure
            
            if total_energy<totals['total_energy']:
                add = True
                cluster_points.append(start)
                totals['num_buildings'] += 1
                totals['SUM_pop'] = totals['SUM_pop']+data.iloc[int(start)]['SUM_pop']
                totals['ELEV_treat'] = min(totals['ELEV_treat'], data.iloc[int(start)]['ELEV_treat'])
                totals['total_dist'] = totals['total_dist'] + piping_distance
                totals['total_energy'] = total_energy
            
            else:
                G.remove_node(int(start))
        if add == False:
            break

In [13]:
len(cluster_points)

60

In [14]:
output = pd.DataFrame()
for i in cluster_points:
    select_row = data.loc[i]
    output = output.append(select_row)

In [15]:
output.to_csv('test_out_5.csv')

In [300]:
totals

{'ELEV_treat': 157.28071499999999,
 'SUM_pop': 12.309001000000002,
 'inbuilding_pumping': 0.0060000000000000001,
 'num_buildings': 5,
 'total_dist': 110.89585263185189,
 'total_energy': 26.299641287943359}